In [ ]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1449, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1449 (delta 27), reused 21 (delta 19), pack-reused 1399
Receiving objects: 100% (1449/1449), 12.76 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (818/818), done.
/content/KaMIS
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
--

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [9]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (Да)#


In [10]:
FILENAME = "NOK6_ISK5"
kv = [0, 504, 63, 455, 658]
tt = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    chk = 0
    for v in kv:
        if dist(v, i) > 6:
            chk = 10000
        elif dist(v, i) == 6:
            chk += 1
    if chk == 3:
        lhs = 0 ^ 945 ^ 910 ^ 630 ^ i
        flag = True
        for v in kv:
            if lhs ^ v == 0:
                flag = False
        if flag:
            tt.append(i)
print(len(tt))
tt

117


[9,
 12,
 29,
 33,
 36,
 43,
 46,
 53,
 65,
 68,
 72,
 95,
 96,
 119,
 123,
 126,
 175,
 189,
 197,
 207,
 231,
 232,
 249,
 252,
 257,
 260,
 264,
 287,
 288,
 311,
 315,
 318,
 323,
 326,
 343,
 344,
 368,
 378,
 389,
 399,
 423,
 424,
 441,
 444,
 459,
 462,
 469,
 473,
 476,
 483,
 486,
 490,
 497,
 500,
 586,
 593,
 594,
 596,
 603,
 606,
 610,
 627,
 649,
 650,
 652,
 657,
 660,
 667,
 669,
 670,
 673,
 674,
 676,
 683,
 686,
 691,
 693,
 694,
 715,
 718,
 723,
 725,
 726,
 729,
 730,
 732,
 739,
 742,
 746,
 753,
 754,
 756,
 778,
 785,
 786,
 788,
 795,
 798,
 802,
 819,
 822,
 851,
 854,
 858,
 882,
 907,
 915,
 917,
 918,
 921,
 922,
 924,
 931,
 934,
 938,
 946,
 948]

In [11]:
print(0b0111111000, 0b0000111111, 0b0111000111, 0b1010010010)

504 63 455 658


In [12]:
base_conf = [0, 945, 910, 630, 219]
abac = []
for i in range(5):
    for j in range(i, 5):
        for q in range(j + 1, 5):
            abac.append((base_conf[i], base_conf[j], base_conf[q]))
abac

[(0, 0, 945),
 (0, 0, 910),
 (0, 0, 630),
 (0, 0, 219),
 (0, 945, 910),
 (0, 945, 630),
 (0, 945, 219),
 (0, 910, 630),
 (0, 910, 219),
 (0, 630, 219),
 (945, 945, 910),
 (945, 945, 630),
 (945, 945, 219),
 (945, 910, 630),
 (945, 910, 219),
 (945, 630, 219),
 (910, 910, 630),
 (910, 910, 219),
 (910, 630, 219),
 (630, 630, 219)]

In [13]:
base_conf = [0, 945, 910, 630, 219]

k2s = [(0, 945),
       (0, 910),
       (0, 630),
       (0, 219),
       (219, 945),
       (219, 910),
       (219, 630),
       (630, 945),
       (630, 910),
       (910, 945)
       ]
kl3s = [(0, 0, 945),
        (0, 0, 910),
        (0, 0, 630),
        (0, 0, 219),
        (0, 945, 910),
        (0, 945, 630),
        (0, 945, 219),
        (0, 910, 630),
        (0, 910, 219),
        (0, 630, 219),
        (945, 945, 910),
        (945, 945, 630),
        (945, 945, 219),
        (945, 910, 630),
        (945, 910, 219),
        (945, 630, 219),
        (910, 910, 630),
        (910, 910, 219),
        (910, 630, 219),
        (630, 630, 219)]
kl4s = [(945, 910, 630, 219), (0, 910, 630, 219), (0, 945, 630, 219), (0, 945, 910, 219), (0, 945, 910, 630)]
kl5s = [(0, 945, 910, 630, 219)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl4 in kl4s:
        fflag = False
        for v in kl4:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

In [14]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K6(base_conf, add_conf, kl5s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl5 in kl5s:
            fflag = False
            for v in kl5:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok6(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], kl5s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl5 in kl5s:
            fflag = False
            for v in kl5:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl5s_updated = []
        for k4ss in kl4s:
            if dist(elem, k4ss[0]) == 6 and dist(elem, k4ss[1]) == 6 and dist(elem, k4ss[2]) == 6 and dist(elem, k4ss[3]) == 6:
                kl5s_updated.append((k4ss[3], k4ss[2], k4ss[0], k4ss[1], elem))
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]) == 6:
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok6(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, kl5s = kl5s + kl5s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K6(base_conf=base_conf, add_conf=add_conf + [elem], kl5s = kl5s + kl5s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok6_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], kl5s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl4 in kl5s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue

        kl5s_updated = []
        for k4ss in kl4s:
            if dist(elem, k4ss[0]) == 6 and dist(elem, k4ss[1]) == 6 and dist(elem, k4ss[2]) == 6 and dist(elem, k4ss[3]) == 6:
                kl5s_updated.append((k4ss[3], k4ss[2], k4ss[0], k4ss[1], elem))
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]) == 6:
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        grph = simple_cook_graph_no_K6(base_conf=base_conf, add_conf=add_conf + [elem], kl5s = kl5s + kl5s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_nok6_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, kl5s = kl5s + kl5s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [16]:
%%time
print("COLORED", COLORSS)
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
bruteforce_nolimits_nok6(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM, 20, 10, 6)
print("AFTER 3-COLORED", COLORSS)

COLORED 5
to check: 60 121
[[0, 0], [45, 58], [0, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [45, 57], [1, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [44, 56], [2, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [54, 55], [3, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [7, 53], [5, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [12, 52], [6, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [19, 51], [7, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [29, 50], [8, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [37, 49], [9, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [46, 48], [10, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [8, 46], [12, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [15, 45], [13, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [27, 44], [14, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [4, 42], [16, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [19, 41], [17, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [37, 40], [18, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [13, 38], [20, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [34, 37], [21, 59], [0, 60], [0, 0], [0, 0]]
[[0, 0], [11,

[[0, 0], [15, 27], [26, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [14, 25], [28, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [0, 22], [31, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [10, 20], [33, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [14, 17], [36, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [8, 13], [40, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [2, 7], [46, 54], [5, 60], [0, 0], [0, 0]]
[[0, 0], [31, 52], [0, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [39, 51], [1, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [10, 49], [3, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [22, 48], [4, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [31, 47], [5, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [8, 45], [7, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [27, 44], [8, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [3, 42], [10, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [17, 41], [11, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [39, 40], [12, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [20, 38], [14, 53], [6, 60], [0, 0], [0, 0]]
[[0, 0], [5, 36], [16, 53], [6, 60], [0, 

[[0, 0], [1, 40], [4, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [28, 39], [5, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [24, 37], [7, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [12, 35], [9, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [7, 33], [11, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [7, 31], [13, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [12, 29], [15, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [24, 27], [17, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [11, 24], [20, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [9, 21], [23, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [5, 17], [27, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [1, 6], [38, 45], [14, 60], [0, 0], [0, 0]]
[[0, 0], [33, 43], [0, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [25, 41], [2, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [13, 39], [4, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [9, 37], [6, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [6, 35], [8, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [3, 33], [10, 44], [15, 60], [0, 0], [0, 0]]
[[0, 0], [1, 31], [12, 44], [

[[0, 0], [5, 11], [17, 29], [30, 60], [0, 0], [0, 0]]
[[0, 0], [0, 3], [25, 29], [30, 60], [0, 0], [0, 0]]
[[0, 0], [17, 26], [1, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [17, 24], [3, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [3, 21], [6, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [12, 19], [8, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [8, 16], [11, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [1, 12], [15, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [0, 6], [21, 28], [31, 60], [0, 0], [0, 0]]
[[0, 0], [3, 25], [1, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [6, 23], [3, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [16, 21], [5, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [6, 18], [8, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [5, 15], [11, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [2, 11], [15, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [1, 4], [22, 27], [32, 60], [0, 0], [0, 0]]
[[0, 0], [16, 24], [1, 26], [33, 60], [0, 0], [0, 0]]
[[0, 0], [20, 22], [3, 26], [33, 60], [0, 0], [0, 0]]
[[0, 0], [11, 19], [6, 26], [33, 60

In [17]:
print("we win")

we win


In [26]:
print(COLORSS)

28079


In [29]:
%%time
BIG_NUM=3
bruteforce_nolimits_nok6_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM - 1, 20, 10, 6)
print("2 finished")
print("AFTER 2-COLORED", COLORSS)
bruteforce_nolimits_nok6_short(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, kl5s, BIG_NUM - 2, 20, 10, 6)
print("1 finished")
print("AFTER 1-COLORED", COLORSS)

[[59, 60], [0, 0], [0, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [49, 59], [0, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [39, 58], [1, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [30, 57], [2, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [24, 56], [3, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [20, 55], [4, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [17, 54], [5, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [13, 53], [6, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [11, 52], [7, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [11, 51], [8, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [12, 50], [9, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [15, 49], [10, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [20, 48], [11, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [26, 47], [12, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [33, 46], [13, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [0, 44], [15, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [10, 43], [16, 60], [0, 0], [0, 0], [59, 60]]
[[59, 60], [19, 42], [17, 60], [0, 0], [0, 0]

In [28]:
print(BIG_NUM)

1


In [23]:
%%time
grph = simple_cook_graph_no_K6(base_conf=base_conf,
                        add_conf=[],
                        kl5s = kl5s,
                        k_2=k_2,
                        k_4=k_4,
                        k_6=[], n=10, k=6)
brb = is_colorable(grph, 11, 20)
brb

CPU times: user 885 ms, sys: 96.3 ms, total: 981 ms
Wall time: 1.17 s


True

In [24]:
print("we won")

we won


In [19]:
%%time
BIG_NUM = 1
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
bruteforce_nolimits_nok5_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, BIG_NUM, 20, 10, 6, 6)
print("AFTER 0-COLORED", COLORSS)

NameError: name 'bruteforce_nolimits_nok5_short' is not defined

In [ ]:
print(0b0111111000, 0b0111000111, 0b0000111111, 0b1010010010)


504 455 63 658


In [ ]:
def get_conf_ls(number, base, add_d, k=6):
    alln = base + add_d
    ln = len(alln)
    ls2 = []
    ls3 = []
    ls4 = []
    ls5 = []
    for i in range(ln):
        for j in range(i+1, ln):
            if dist(alln[i], alln[j]) == k:
                ls2.append((alln[i], alln[j]))
    for i in range(ln):
        for j in range(i+1, ln):
            if dist(alln[i], alln[j]) == k:
                for q in range(j+1, ln):
                    if dist(alln[i], alln[q]) == k and dist(alln[j], alln[q]) == k:
                    ls3.append((alln[i], alln[j], alln[q]))
    
    for i in range(ln):
        for j in range(i+1, ln):
            if dist(alln[i], alln[j]) == k:
                for q in range(j+1, ln):
                    if dist(alln[i], alln[q]) == k and dist(alln[j], alln[q]) == k:
                        for q4 in range(q+1, ln):
                            if dist(alln[i], alln[q4]) == k and dist(alln[j], alln[q4]) == k and dist(alln[q], alln[q4]) == k:
                                ls4.append((alln[i], alln[j], alln[q], alln[q4]))
    
    for i in range(ln):
        for j in range(i+1, ln):
            if dist(alln[i], alln[j]) == k:
                for q in range(j+1, ln):
                    if dist(alln[i], alln[q]) == k and dist(alln[j], alln[q]) == k:
                        for q4 in range(q+1, ln):
                            if dist(alln[i], alln[q4]) == k and dist(alln[j], alln[q4]) == k and dist(alln[q], alln[q4]) == k:
                                for q5 in range(q4 + 1, ln):
                                    if dist(alln[i], alln[q5]) == k and dist(alln[j], alln[q5]) == k and dist(alln[q], alln[q5]) == k and dist(alln[q4], alln[q5]) == k:
                                        ls5.append((alln[i], alln[j], alln[q], alln[q4], alln[q5]))
    
    return ls2, ls3, ls4, ls5

0b11110001001000000


In [33]:
%%time
t = 0
for i in range(1000):
    for j in range(1000):
        t += 1

CPU times: user 684 ms, sys: 0 ns, total: 684 ms
Wall time: 1.32 s
